In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer
import seaborn as sns
from sklearn import preprocessing, metrics, mixture
from sklearn.cluster import KMeans
df['group'] = ['SC', 'CL', 'SM', 'CL', 'SM', 'SM', 'SM', 'CL', 'CL', 'SC', 'CL', 'CL', 'CL', 'CL', 'CL']
sns.set()
data = df.iloc[:, np.r_[4:17,20,21,25]].copy()
soil_groups = df.iloc[:,44]
depth = df['Độ sâu lấy mẫu']
data.fillna(0, inplace=True)
scaler = preprocessing.MinMaxScaler()
features_normal = scaler.fit_transform(data)
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,15), timings=True)
visualizer.fit(features_normal)
visualizer.show()

kmeans = KMeans(n_clusters=visualizer.elbow_value_).fit(features_normal)
kmeans_predict = pd.DataFrame(kmeans.labels_).iloc[:,0]
empty_column = np.full((len(soil_groups),1), np.inf)
new_columns = ['DEPTH','SOIL_GROUP', 'KMEANS_PRED', 'KMEANS_GROUP']
new_values = [depth,soil_groups, kmeans_predict, empty_column]

result_mapping = dict(zip(new_columns, new_values))
labeled_features = data[:]
labeled_features = labeled_features.assign(**result_mapping)
label_list = labeled_features['KMEANS_PRED'].unique()
for group in label_list:
	group_name=labeled_features[labeled_features['KMEANS_PRED']==group]['SOIL_GROUP'].mode()[0]
	labeled_features.loc[labeled_features['KMEANS_PRED']==group,'KMEANS_GROUP']=group_name
labeled_features.to_csv( 'kmean_1.csv', encoding='utf-8-sig')

count = []
for i in range(len(labeled_features['KMEANS_GROUP'])):
    if labeled_features["KMEANS_GROUP"][i] == labeled_features["SOIL_GROUP"][i]:
        count.append(1)

len(count)
print('Mức độ chính xác của phương pháp KMeans là: ' + str(len(count)/len(labeled_features['KMEANS_GROUP'])*100))